In [21]:
using Requests, JSON, LightXML

INFO: Recompiling stale cache file /Users/rdeits/.julia/lib/v0.5/LightXML.ji for module LightXML.


In [13]:
response = get("https://en.wikipedia.org/w/api.php", 
query=Dict(
"action" => "query",
"titles" => "List of common fish names",
"prop" => "revisions",
"rvprop" => "content",
"format" => "json"
))

Response(200 OK, 25 headers, 27448 bytes in body)

In [14]:
j = Requests.json(response)
j["query"]["pages"]["330482"]["revisions"][1]["*"];

In [6]:
include("src/caching.jl")
include("test/caching.jl");

running bar


cachingtest.Foo(Nullable{Int64}(),Nullable{Float64}())
running baz
Test Summary: | Pass  Total
  caching     |    6      6


In [7]:
module wiki

using Requests, JSON, Caching

type MediaWiki
    url::String
end

wikipedia = MediaWiki("https://en.wikipedia.org/w/api.php")

function wiki_request(wiki::MediaWiki, query_params)
    query = Dict(
        "format" => "json",
        "action" => "query",
        query_params...
    )
    Requests.json(get(
        wiki.url,
        query=query
    ))
end

type WikiPage
    title::String
    wiki::MediaWiki
    links::Nullable{Vector{String}}
    
    WikiPage(title, wiki=wikipedia) = new(title, 
        wiki, 
        Nullable{Vector{String}}()
    )
end


"""
Based on https://www.mediawiki.org/wiki/API:Query#Continuing_queries
"""
function continued_query(page::WikiPage, query_params)
    continuation = Dict{String, String}()
    prop = query_params["prop"]
    results = []
    
    while true
        response = wiki_request(page.wiki, Dict(
            "titles" => page.title,
            continuation...,
            query_params...
        ))
        for (page_id, page_data) in response["query"]["pages"]
            append!(results, page_data[prop])
        end
        if "continue" in keys(response)
            continuation = response["continue"]
        else
            break
        end
    end
    results
end

@cached function links(page::WikiPage)
    raw_links = continued_query(page, Dict(
    "prop" => "links",
    "plnamespace" => 0,
    "pllimit" => "max",
    ))
    String[d["title"] for d in raw_links]
end
        
end

wiki

In [8]:
# page = wiki.WikiPage("List of common fish names")
page = wiki.WikiPage("Novelty and fad dances")

wiki.WikiPage("Novelty and fad dances",wiki.MediaWiki("https://en.wikipedia.org/w/api.php"),Nullable{Array{String,1}}())

In [11]:
links = wiki.links(page)

224-element Array{String,1}:
 "Achy Breaky Heart"      
 "Acro dance"             
 "African-American dance" 
 "African dance"          
 "Algorithm March"        
 "Allemande"              
 "Assyrian folk dance"    
 "B-boying"               
 "Bach"                   
 "Ballet"                 
 "Ballet technique"       
 "Ballroom dance"         
 "Batusi"                 
 ⋮                        
 "Twerking"               
 "Twist (dance)"          
 "Ukrainian dance"        
 "Vogue (Madonna song)"   
 "Walk like an Egyptian"  
 "Waltz"                  
 "War dance"              
 "Watch Me (Whip/Nae Nae)"
 "Watusi (dance)"         
 "Whip My Hair"           
 "Women in dance"         
 "Y.M.C.A. (song)"        